In [1]:
import re
import qiskit
import json
import time

import numpy as np
from typing import List
from qiskit import transpile
import time

from fast_generator import fc_tree_commute_recur_lookahead_fast

from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
from circuit_generator import construct_qcc_circuit

import numpy as np
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from utilities import load_sycamore_coupling_map


In [2]:
#Compare json files in a specific folder
import os, json
def run_experiment_folder(folder_path = None, filename = None, save_output = False, threshold = 1):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
            end_time = time.time()
            combined_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0)
                },
                "circuit_entangling_depth": {
                    "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1)
                },
                "test_paulis_file": f'benchmarks/results_hardware_topology_google_sycamore/test_new_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results_hardware_topology_google_sycamore/test_new_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [3]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, threshold = 1, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=load_sycamore_coupling_map())
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0)
        },
        "circuit_entangling_depth": {
            "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1)
        },
        "test_paulis_file": f'benchmarks/results_hardware_topology_google_sycamore/test_new_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results_hardware_topology_google_sycamore/test_new_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [4]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,]# 12]
orbitals_list = [4, 6, 8, 12, 16, 20,]# 24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = True, threshold = 1, filename=f"Paulis{len(test_paulis)}.json")

{'num_paulis': 24, 'times': {'our_time': 0.3919501304626465, 'combined_time': 0.19647836685180664}, 'gate_counts': {'our_method': 63, 'combined_method': 32}, 'circuit_entangling_depth': {'our_method': 45, 'combined_method': 25}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_Paulis24.json'}
{'num_paulis': 80, 'times': {'our_time': 1.0811119079589844, 'combined_time': 1.1479313373565674}, 'gate_counts': {'our_method': 416, 'combined_method': 157}, 'circuit_entangling_depth': {'our_method': 293, 'combined_method': 113}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_Paulis80.json'}
{'num_paulis': 320, 'times': {'our_time': 4.59771466255188, 'combined_time': 3.828770399093628}, 'gate_counts': {'our_method': 1452, 'combined_method': 785}, 'circuit_entangling_depth': {'our_method': 984, 'combined_method': 565}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_Paulis320.json'}
{'num_paul

In [5]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True, threshold = 1)

benzene.json
{'num_paulis': 1254, 'times': {'our_time': 36.50245928764343, 'combined_time': 28.916728734970093}, 'gate_counts': {'our_method': 10624, 'combined_method': 6285}, 'circuit_entangling_depth': {'our_method': 5677, 'combined_method': 4223}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_benzene.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 4.105029344558716, 'combined_time': 2.2958645820617676}, 'gate_counts': {'our_method': 1213, 'combined_method': 537}, 'circuit_entangling_depth': {'our_method': 737, 'combined_method': 400}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_H2O.json'}
LiH.json
{'num_paulis': 61, 'times': {'our_time': 1.282571792602539, 'combined_time': 0.7878942489624023}, 'gate_counts': {'our_method': 307, 'combined_method': 137}, 'circuit_entangling_depth': {'our_method': 195, 'combined_method': 124}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/t

In [6]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 1)

max_cut_benchmark_connected_n10_e12_l1.json
{'num_paulis': 22, 'times': {'our_time': 0.5694773197174072, 'combined_time': 0.3869662284851074}, 'gate_counts': {'our_method': 227, 'combined_method': 44}, 'circuit_entangling_depth': {'our_method': 96, 'combined_method': 35}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_max_cut_benchmark_connected_n10_e12_l1.json'}
max_cut_benchmark_connected_n10_e12_l3.json
{'num_paulis': 66, 'times': {'our_time': 1.32545804977417, 'combined_time': 1.0731322765350342}, 'gate_counts': {'our_method': 515, 'combined_method': 133}, 'circuit_entangling_depth': {'our_method': 247, 'combined_method': 92}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_max_cut_benchmark_connected_n10_e12_l3.json'}
max_cut_benchmark_connected_n15_e63_l1.json
{'num_paulis': 78, 'times': {'our_time': 5.138269662857056, 'combined_time': 4.646583557128906}, 'gate_counts': {'our_method': 590, 'combined_method': 1

In [9]:
#since in the maxcut problems there are many commuting terms and it's slow to search through all of them,
#we can set the threshold to 2 to early stop the search once found a simplified pauli of weight 2
#you can see the run time difference, for the second benchmark runtime reduces from 30second to 7 seconds.

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 2)

max_cut_benchmark_connected_n10_e12_l1.json
{'num_paulis': 22, 'times': {'our_time': 0.485701322555542, 'combined_time': 0.31316423416137695}, 'gate_counts': {'our_method': 248, 'combined_method': 44}, 'circuit_entangling_depth': {'our_method': 117, 'combined_method': 38}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_max_cut_benchmark_connected_n10_e12_l1.json'}
max_cut_benchmark_connected_n10_e12_l3.json
{'num_paulis': 66, 'times': {'our_time': 1.145937442779541, 'combined_time': 0.9145586490631104}, 'gate_counts': {'our_method': 446, 'combined_method': 120}, 'circuit_entangling_depth': {'our_method': 206, 'combined_method': 85}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_max_cut_benchmark_connected_n10_e12_l3.json'}
max_cut_benchmark_connected_n15_e63_l1.json
{'num_paulis': 78, 'times': {'our_time': 3.6422665119171143, 'combined_time': 2.8922715187072754}, 'gate_counts': {'our_method': 642, 'combined_method

In [8]:
#Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = True, threshold= 2)

labs_n10_layers1.json
{'num_paulis': 80, 'times': {'our_time': 2.759625196456909, 'combined_time': 1.896932601928711}, 'gate_counts': {'our_method': 628, 'combined_method': 248}, 'circuit_entangling_depth': {'our_method': 321, 'combined_method': 173}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_labs_n10_layers1.json'}
labs_n10_layers3.json
{'num_paulis': 240, 'times': {'our_time': 6.822770118713379, 'combined_time': 5.197111129760742}, 'gate_counts': {'our_method': 1475, 'combined_method': 743}, 'circuit_entangling_depth': {'our_method': 783, 'combined_method': 519}, 'test_paulis_file': 'benchmarks/results_hardware_topology_google_sycamore/test_new_labs_n10_layers3.json'}
labs_n11_layers1.json
{'num_paulis': 106, 'times': {'our_time': 4.055164575576782, 'combined_time': 3.300180435180664}, 'gate_counts': {'our_method': 731, 'combined_method': 357}, 'circuit_entangling_depth': {'our_method': 383, 'combined_method': 252}, 'test_paulis_file': 'benchm